# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [58]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [ ]:
#!pip install pandas

In [31]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [32]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [81]:
import os
from langchain.llms import CTransformers
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

In [82]:
MODEL_PATH = 'mistral-7b-instruct-v0.2.Q4_K_M.gguf'

config = {
    "max_new_tokens": 2048,
    "context_length": 4096,
    "repetition_penalty": 1.1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2)
}

In [83]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [84]:
llm = CTransformers(model=MODEL_PATH, config=config, callback_manager=callback_manager)

In [ ]:
# llm = LlamaCpp(
#     model_path="mistral-7b-instruct-v0.2.Q4_K_M.gguf",
#     # model_path="Meta-Llama-3-8B-Instruct-Q6_K.gguf",    
#     temperature=0.50,
#     max_tokens=2000,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=True,  # Verbose is required to pass to the callback manager
# )

In [103]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [104]:
chain = LLMChain(llm=llm, prompt=prompt)

In [105]:
product = "Queen Size Sheet Set"
chain.run(product)



A: A company that specializes in producing and selling queen size sheet sets can be named as follows:

1. QueenBedLinens Co.
2. MonarchSheets Inc.
3. RegalRestCo.
4. GrandeBedding Co.
5. RoyalSlumber LLC.
6. MajesticSheets Factory.
7. PalaceLinen Corporation.
8. PremiumQueenBeddings.
9. EliteSheetMakers.
10. LuxeQueenLinens Ltd.

These names reflect the product line (queen size sheets) and evoke a sense of luxury or exclusivity that consumers often associate with high-quality bedding products.

'\n\nA: A company that specializes in producing and selling queen size sheet sets can be named as follows:\n\n1. QueenBedLinens Co.\n2. MonarchSheets Inc.\n3. RegalRestCo.\n4. GrandeBedding Co.\n5. RoyalSlumber LLC.\n6. MajesticSheets Factory.\n7. PalaceLinen Corporation.\n8. PremiumQueenBeddings.\n9. EliteSheetMakers.\n10. LuxeQueenLinens Ltd.\n\nThese names reflect the product line (queen size sheets) and evoke a sense of luxury or exclusivity that consumers often associate with high-quality bedding products.'

## SimpleSequentialChain

In [106]:
from langchain.chains import SimpleSequentialChain

In [107]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [108]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [109]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], 
    verbose=True
)

In [110]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...


A: A company that specializes in producing and selling queen size sheet sets could be described as follows:

1. Queen Size Linen Company
2. Comfortable Queen Sheets Producer
3. Regal Rest Sheets Manufacturer
4. Queen Bedding Solutions
5. Monarch Mattress Accessories
6. Luxe Queen Sheet Set Provider
7. Royal Rest Company
8. Grand Queen Linens
9. Serene Queen Sheet Maker
10. Tranquil Queen Bedding Co.

A: A company that specializes in producing and selling queen size sheet sets could be described as follows:

1. Queen Size Linen Company
2. Comfortable Queen Sheets Producer
3. Regal Rest Sheets Manufacturer
4. Queen Bedding Solutions
5. Monarch Mattress Accessories
6. Luxe Queen Sheet Set Provider
7. Royal Rest Company
8. Grand Queen Linens
9. Serene Queen Sheet Maker
10. Tranquil Queen Bedding Co.

11. Majestic Queen Sheets Supplier
12. Premium Queen Linen Firm
13. Noble Queen Sheets Producer
14. Dream Queen Linen Supplies
15. Elegant Que

'\n11. Majestic Queen Sheets Supplier\n12. Premium Queen Linen Firm\n13. Noble Queen Sheets Producer\n14. Dream Queen Linen Supplies\n15. Elegant Queen Sheet Set Creator\n16. Opulent Queen Bedding Co.\n17. Lavish Queen Linens\n18. Deluxe Queen Sheet Vendor\n19. Palatial Queen Sheets Manufacturer\n20. Plush Queen Bedding Emporium'

## SequentialChain

In [85]:
from langchain.chains import SequentialChain

In [86]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review" )


In [87]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary" )


In [88]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(
    llm=llm, 
    prompt=third_prompt,
    output_key="language"
)


In [89]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=llm, 
    prompt=fourth_prompt,
    output_key="followup_message"
)


In [90]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [91]:
review = df.Review[5]
review

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [92]:
overall_chain(review)



> Entering new SequentialChain chain...


I find the taste mediocre. The foam doesn't hold up, that's strange. I buy the same ones in the store and the taste is much better... Old batch or counterfeit!?

A customer expresses dissatisfaction with the product's taste being mediocre and the poor durability of its foam, suspecting it to be an old or counterfeit batch.

This review is in French. The person is expressing that they find the taste mediocre and the mousse doesn't hold up well. They also mention that they buy the same product elsewhere and it tastes better, questioning whether it might be an old batch or a counterfeit.

Response: 

Nous sommes désolés d'apprendre que vous n'êtes pas satisfait de la saveur médiocre et de la faible durée de la mousse de notre produit. Nous comprenons vos inquiétudes quant à l'authenticité ou à une possible livraison hâtive de ce lot, mais nous voulons vous assurer que tous nos lots sont contrôlés et certifiés avant expédition. Si cela est le cas

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "\n\nI find the taste mediocre. The foam doesn't hold up, that's strange. I buy the same ones in the store and the taste is much better... Old batch or counterfeit!?",
 'summary': "\n\nA customer expresses dissatisfaction with the product's taste being mediocre and the poor durability of its foam, suspecting it to be an old or counterfeit batch.",
 'followup_message': "\n\nResponse: \n\nNous sommes désolés d'apprendre que vous n'êtes pas satisfait de la saveur médiocre et de la faible durée de la mousse de notre produit. Nous comprenons vos inquiétudes quant à l'authenticité ou à une possible livraison hâtive de ce lot, mais nous voulons vous assurer que tous nos lots sont contrôlés et certifiés avant expédition. Si cela est le cas, nous vous invitons à contacter notre service client pour obt

## Router Chain

In [93]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [94]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [95]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [96]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [97]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [98]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [99]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [100]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [101]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [102]:
chain.run("What is the story about Ireland?")



> Entering new MultiPromptChain chain...

{
    "destination": "History",
    "next_inputs": "What is the historical significance of Ireland?"
}
```
The original input "What is the story about ireland?" can be revised to "What is the historical significance of Ireland?" in order to better suit the History prompt. This will likely lead to a more accurate and informative response from the language model.History: {'input': 'What is the historical significance of Ireland?'} How has this small island nation shaped the world in various aspects such as politics, economy, culture, and society?

A: Ireland, an emerald-shaped island located at the northwestern edge of Europe, has played a significant role in shaping the world throughout history, despite its relatively small size. Ireland's impact can be seen in various domains, including politics, economy, culture, and society.

1. Politics: Ireland's political influence began with its strategic location between Europe and North America. Its p

' How has this small island nation shaped the world in various aspects such as politics, economy, culture, and society?\n\nA: Ireland, an emerald-shaped island located at the northwestern edge of Europe, has played a significant role in shaping the world throughout history, despite its relatively small size. Ireland\'s impact can be seen in various domains, including politics, economy, culture, and society.\n\n1. Politics: Ireland\'s political influence began with its strategic location between Europe and North America. Its position made it a valuable asset during the colonial era, as evidenced by England\'s desire to control Ireland for its proximity to English interests in North America. The struggle for Irish independence and the subsequent peace processes have also influenced global politics, particularly in the areas of decolonization and conflict resolution.\n\n2. Economy: Ireland has significantly impacted the global economy through its economic successes and challenges. The Gre

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

Reminder: Download your notebook to you local computer to save your work.

In [57]:
chain.run("tell me about Dell in 2 bullet points")



> Entering new MultiPromptChain chain...

```json
{
    "destination": "DEFAULT",
    "next_inputs": "Dell is an American multinational computer technology company. Its primary hardware products include desktop computers, laptops, data servers, network switches, and storage devices.\n\n- Dell Inc. was founded in 1984 by Michael Dell.\n- The company is known for its direct sales model: Dell sells personal computers (PCs) directly to customers, eliminating middlemen and reducing costs."
}
```None: {'input': 'Dell is an American multinational computer technology company. Its primary hardware products include desktop computers, laptops, data servers, network switches, and storage devices.\n\n- Dell Inc. was founded in 1984 by Michael Dell.\n- The company is known for its direct sales model: Dell sells personal computers (PCs) directly to customers, eliminating middlemen and reducing costs.'} This business model has been a significant factor in Dell's success.
- In addition to hardware, D

" This business model has been a significant factor in Dell's success.\n- In addition to hardware, Dell offers software, peripherals, and technical support services. It also provides IT consulting services through its Dell Services division.\n- Dell is headquartered in Round Rock, Texas, and employs more than 103,000 people worldwide.\n- The company's revenue for the fiscal year 2020 was approximately $92.2 billion.\n- Dell has a significant presence in the global market, with operations in over 180 countries. It serves customers in various industries, including education, finance services, healthcare and life sciences, information technology and telecommunications, retail, manufacturing, public sector, and energy resources."